In [1]:
import plotly.express as px
import numpy as np
import pandas as pd
import DataClean
import datetime
import os
# import plotly.graph_objects as go

In [24]:
import pickle

In [25]:
dat = pickle.load(open('compiled_data.p', 'rb'))
countries = list(dat.index.get_level_values(0).unique())
stats = list(dat.columns)
stat_map = {
    'DailyPosTestRate': 'Daily Positive Test Rate',
    'Confirmed': 'Confirmed Tests',
    'NewTests': 'Daily New Tests',
    'CumulativePosTestRate': 'Cumulative Positive Test Rate',
    'NewCases': 'New Cases',
    'TotalTests': 'Total Tests',
    'ConfirmedGrowth': 'Daily Confirmed Cases Growth (%)',
    'CumulativeDeathRate': 'Cumulative Death Rate',
    'HospitalizationRate': 'Hospitalization Rate',
    'TotalTestsPer10k': 'Total Tests per 10k Population',
    'ConfirmedPer10k': 'Total Confirmed per 10k Population',
    'NewTestsPer10k': 'Daily Tests per 10k Population'
}

xs = ['DaysSinceFirst', 'Date', 'DaysSinceTenthDeath', 'DaysSinceShutdown']
xs_map = {'DaysSinceFirst': 'Days Since First Confirmed',
         'DaysSinceTenthDeath': 'Days Since 10th Death',
         'DaysSinceShutdown': 'Days Since Economic Shutdown'}

# Important Dates by Country:

### Peru
- Sourhces: ministerio de salud, wikipedia page
* March 11 - Declaran emergencia sanitaria por 90 dias
* March 13 - Suspenden vuelos internacional desde Europa y Asia
* March 16 - Teletrabajo para todos menos los esenciales
* March 17 - Cierran fronteras
* March 18 - Cuarentena general entre 8PM y 5 AM.
* https://www.dge.gob.pe/portal/

### Panama
* Something

### Guatemala
- Source: Alejandro Giammattei twitter
* March 22 - Cuarentena general entre 4 PM y 4 AM por 8 dias

### Testing Analysis stuff

In [170]:
import requests
from sodapy import Socrata
import pandas as pd
import numpy as np
import datetime

In [171]:
client = Socrata("datos.gov.co",
                 "aSEItRti2CveuMdUcoBfyeWTQ",
                )
#                  user = "abishalom",
#                  password = "PAi#%4NHPQhu4z")

results = client.get("gt2j-8ykr", limit = 100000)

results = pd.DataFrame.from_records(results)

# results

In [172]:
#filter out babies that have ages in months
results = results[~results.edad.str.contains('mes')]

new_map = {
    'fecha_de_diagn_stico': 'fecha_de_diagnostico',
    'ciudad_de_ubicaci_n' : 'ciudad_de_ubicacion',
    'atenci_n': 'atencion',
    'pa_s_de_procedencia': ' pais_de_procedencia'
}
results = results.rename(columns = new_map)
results.fecha_de_diagnostico = pd.to_datetime(results.fecha_de_diagnostico)
results.fecha_recuperado = results.fecha_recuperado.replace({'No': np.nan})
results.fecha_recuperado = pd.to_datetime(results.fecha_recuperado)
results.fis = results.fis.replace(['Asintomático', 'No disponible'], np.nan)
results.fis = pd.to_datetime(results.fis)
results.fecha_de_muerte = pd.to_datetime(results.fecha_de_muerte)

results.id_de_caso = results.id_de_caso.astype('int32')
results.edad = results.edad.astype('int32')

results = results.set_index("id_de_caso")

In [177]:
results = results.rename(columns = {'atencion' : datetime.date.today()})

In [208]:
old = pd.read_csv('data/Colombia_data.csv', index_col = 0, parse_dates = True)
old = old[:5]
results = results[:5]

In [209]:
# results.iloc[1].fecha_de_muerte = datetime.date.today()
# results.at[1, 'fecha_de_muerte'] = datetime.date.today()

results

,fecha_de_diagnostico,ciudad_de_ubicacion,departamento,2020-04-19,edad,sexo,tipo,pais_de_procedencia,fecha_recuperado,fis,fecha_de_muerte
id_de_caso,,,,,,,,,,,
1,2020-03-06,Bogota,Bogotá D.C.,Recuperado,19,F,Importado,Italia,2020-03-13,2020-02-27,2020-04-19
2,2020-03-09,Buga,Valle del Cauca,Recuperado,34,M,Importado,España,2020-03-19,2020-03-04,NaT
3,2020-03-09,Medellin,Antioquia,Recuperado,50,F,Importado,España,2020-03-15,2020-02-29,NaT
4,2020-03-11,Medellin,Antioquia,Recuperado,55,M,Relacionado,Colombia,2020-03-26,2020-03-06,NaT
5,2020-03-11,Medellin,Antioquia,Recuperado,25,M,Relacionado,Colombia,2020-03-23,2020-03-08,NaT


In [210]:
old

,fecha_de_diagnostico,ciudad_de_ubicacion,departamento,2020-04-18,edad,sexo,tipo,pais_de_procedencia,fecha_recuperado,fis,fecha_de_muerte
id_de_caso,,,,,,,,,,,
1,2020-03-06 00:00:00,Bogota,Bogotá D.C.,Recuperado,19,F,Importado,Italia,2020-03-13 00:00:00,2020-02-27 00:00:00,NaN
2,2020-03-09 00:00:00,Buga,Valle del Cauca,Recuperado,34,M,Importado,España,2020-03-19 00:00:00,2020-03-04 00:00:00,NaN
3,2020-03-09 00:00:00,Medellin,Antioquia,Recuperado,50,F,Importado,España,2020-03-15 00:00:00,2020-02-29 00:00:00,NaN
4,2020-03-11 00:00:00,Medellin,Antioquia,Recuperado,55,M,Relacionado,Colombia,2020-03-26 00:00:00,2020-03-06 00:00:00,NaN
5,2020-03-11 00:00:00,Medellin,Antioquia,Recuperado,25,M,Relacionado,Colombia,2020-03-23 00:00:00,2020-03-08 00:00:00,NaN


In [218]:
x = old.combine_first(results)

def mixs(x):
    try:
        x_date = pd.to_datetime(x)
        return (1, x_date, '')
    except:
        return (0, x, '')

# sorted(x.columns, key = mixs)
x.reindex(sorted(x.columns, key = mixs), axis=1)

,pais_de_procedencia,ciudad_de_ubicacion,departamento,edad,fecha_de_diagnostico,fecha_de_muerte,fecha_recuperado,fis,sexo,tipo,2020-04-18,2020-04-19
id_de_caso,,,,,,,,,,,,
1,Italia,Bogota,Bogotá D.C.,19,2020-03-06 00:00:00,2020-04-19,2020-03-13 00:00:00,2020-02-27 00:00:00,F,Importado,Recuperado,Recuperado
2,España,Buga,Valle del Cauca,34,2020-03-09 00:00:00,NaT,2020-03-19 00:00:00,2020-03-04 00:00:00,M,Importado,Recuperado,Recuperado
3,España,Medellin,Antioquia,50,2020-03-09 00:00:00,NaT,2020-03-15 00:00:00,2020-02-29 00:00:00,F,Importado,Recuperado,Recuperado
4,Colombia,Medellin,Antioquia,55,2020-03-11 00:00:00,NaT,2020-03-26 00:00:00,2020-03-06 00:00:00,M,Relacionado,Recuperado,Recuperado
5,Colombia,Medellin,Antioquia,25,2020-03-11 00:00:00,NaT,2020-03-23 00:00:00,2020-03-08 00:00:00,M,Relacionado,Recuperado,Recuperado


In [214]:
# isinstance(datetime.date.today(), datetime.date)
pd.to_datetime(datetime.date.today())

Timestamp('2020-04-19 00:00:00')